In [148]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel, BertConfig
import torch.utils.data as data_utils
#from pytorch_pretrained_bert import BertTokenizer, BertModel, BertConfig
# def pad_sequence(sequence,max_length):
#     if len(sequence)<max_length:
#         sequence += (max_length-len(sequence))*[0]
#     return sequence

def embeddings_from_dataset(X, tokenizer, bert_model):

    X = [tokenizer.tokenize('[CLS] ' + sent + ' [SEP]') for sent in X] # Appending [CLS] and [SEP] tokens - this probably can be done in a cleaner way
    #X_test = [tokenizer.tokenize('[CLS] ' + sent + ' [SEP]') for sent in X_test] # Appending [CLS] and [SEP] tokens - this probably can be done in a cleaner way
    X = [text[:512] if len(text)>512 else text for text in X]
    #X_test = [text[:512] if len(text)>512 else text for text in X_test]
    X_tokens = [tokenizer.convert_tokens_to_ids(sent) for sent in X]
    #X_tokens = [pad_sequence(sequence) if len(sequence)<512 else sequence for sequence in X_tokens]

    #X_test_tokens = [tokenizer.convert_tokens_to_ids(sent) for sent in X_test]

    train_embeddings = []
    #test_embeddings = []

    #results = torch.zeros((len(X_train_tokens), bert_model.config.hidden_size)).long()
    with torch.no_grad():
        for stidx in range(len(X_tokens)):
            tokens = X_tokens[stidx]
            tokens_t = torch.LongTensor(tokens)#.to(device)
            segment_t = torch.LongTensor([1] * len(tokens))#.to(device)
            outputs = bert_model(tokens_t.unsqueeze(0),segment_t.unsqueeze(0))
            embeddings = outputs[0][0][0] #This only takes CLS embedding
            train_embeddings.append(embeddings.cpu())
            #results[stidx] = embeddings.cpu()
        
        # for stidx in range(len(X_test)):
        #     tokens = X_test_tokens[stidx]
        #     tokens_t = torch.LongTensor(tokens)#.to(device)
        #     segment_t = torch.LongTensor([1] * len(tokens))#.to(device)
        #     outputs = bert_model(tokens_t.unsqueeze(0),segment_t.unsqueeze(0))
        #     embeddings = outputs[0][0][0] #This only takes CLS embedding
        #     test_embeddings.append(embeddings.cpu())
    return torch.stack(train_embeddings)

def train_model(model,optimizer,criterion,loader):
    model.train()
    total_loss = 0
    for data in loader:
        x = data[0]
        label = data[1]
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred,label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    return total_loss/len(loader.dataset)    

def test_model(model,criterion,loader):
    model.eval()
    total_loss = 0
    for data in loader:
        x = data[0]
        label = data[1]
        pred = model(x)
        loss = criterion(pred,label)
        total_loss += loss.item()
    
    return total_loss/len(loader.dataset)
class BertClassifier(nn.Module):
    def __init__(self, input_size = 768):
        super().__init__()
        self.fc1 = nn.Linear(input_size,100)
        self.fc2 = nn.Linear(100,50)
        self.fc3 = nn.Linear(50,1)

    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))

        return x



corpus_reddit = pd.read_csv('reddit_corpus_agree.csv')
corpus_reddit['label'] = 0
corpus_reddit['label'].loc[(corpus_reddit['cls']=='Risk')] = 1 
del corpus_reddit['cls']

batch_size = 1
texts, labels =  corpus_reddit['text'], corpus_reddit['label']
#texts = [" ".join(text.split()[:512]) if len(text.split())>512 else text for text in texts]


X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.1, stratify=corpus_reddit['label'])
model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
config = BertConfig.from_pretrained(model_name, output_hidden_states=True)    
bert_model = BertModel.from_pretrained(model_name, config=config)
bert_model = bert_model#.to(device)
bert_model.eval()

train_embeddings = embeddings_from_dataset(X_train, tokenizer, bert_model)
test_embeddings = embeddings_from_dataset(X_test, tokenizer, bert_model)

train_dataset = data_utils.TensorDataset(train_embeddings , torch.FloatTensor(y_train.values).view(-1,1))
test_dataset = data_utils.TensorDataset(test_embeddings , torch.FloatTensor(y_test.values).view(-1,1))

train_loader = data_utils.DataLoader(train_dataset,batch_size=10,shuffle=True)
test_loader = data_utils.DataLoader(test_dataset,batch_size=1)

my_NN = BertClassifier()
optimizer = torch.optim.Adam(my_NN.parameters(),lr=0.001)
criterion = nn.BCELoss()

for epoch in range(10):
    loss = train_model(my_NN,optimizer,criterion,train_loader)
    print(f"Epoch {epoch+1}, train_loss: {loss}")
    test_loss = test_model(my_NN,criterion,test_loader)
    print(f"test_loss: {test_loss}")



C:\Users\ASD\anaconda3\envs\pytorchenv\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


Epoch 1, train_loss: 0.044756897329504976
test_loss: 0.3842197222014268
Epoch 2, train_loss: 0.03492586689642052
test_loss: 0.36963170228732956
Epoch 3, train_loss: 0.03325567696413963
test_loss: 0.367331893907653
Epoch 4, train_loss: 0.03133594176543304
test_loss: 0.369563566106889
Epoch 5, train_loss: 0.0289524326624434
test_loss: 0.37096358297599685
Epoch 6, train_loss: 0.027393649947019965
test_loss: 0.3730640320314301
Epoch 7, train_loss: 0.025156307186371362
test_loss: 0.38547680821890634
Epoch 8, train_loss: 0.021222296712640063
test_loss: 0.46947262845868765
Epoch 9, train_loss: 0.02108464575086544
test_loss: 0.44884899545448004
Epoch 10, train_loss: 0.016937596626044098
test_loss: 0.49029716747301993


In [157]:
deneme = embeddings_from_dataset([X_train[0]],tokenizer,bert_model)
with torch.no_grad():
    BertClassifier(deneme)

TypeError: new(): argument 'size' must be tuple of ints, but found element of type Tensor at pos 2

In [160]:
my_NN(deneme)

tensor([[0.9921]], grad_fn=<SigmoidBackward>)

In [163]:
list(y_train)

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1]

In [185]:

deneme = embeddings_from_dataset([X_test.iloc[-6]],tokenizer,bert_model)
with torch.no_grad():
    print(my_NN(deneme))



tensor([[0.9769]])


In [183]:
y_test

44     1
94     1
56     1
9      1
75     1
144    1
170    1
61     1
31     1
163    1
58     1
70     1
111    0
22     1
27     0
93     1
26     1
127    1
Name: label, dtype: int64

120    1
54     1
48     1
159    1
135    1
      ..
145    0
59     1
32     1
62     0
24     1
Name: label, Length: 153, dtype: int64

In [173]:
X_train.values[22]

'My secret suicidal thoughts are getting worse. It\'s getting harder to disobey. And I dont want to tell anyone\r\nI\'m one of those super annoying people pleaser types. I\'m constantly doing favors. Listening to other people\'s problems. And forever smiling and trying to keep the positive vibes going.\r\n\r\nI\'m worse than just all that though. Because im an airhead and I dont have good social skills. I\'m always saying or doing something stupid. And it makes me more awkward. I act like it\'s all good and I\'m happy and it doesnt phase me. But that is so far from the truth. \r\n\r\nEvery single day I\'m plagued by thoughts of how weak I am and how I\'m annoying and how I\'m a bitch and "I need to die" over and over and over. Which turns into "I wanna kill myself" over and over. It\'s getting worse every day. I dont expect help. I just needed a place to vent.'

In [172]:
y_train.values[22]

1